In [4]:
import argparse
import os

import requests

S2_API_KEY = os.getenv('S2_KEY')


def main():   
   
    paper_id='ac8efed448c7096f3e288ce738260384df9a9f2c'
   
    paper = get_paper(paper_id)
    print(f'"{paper["title"]}" has {len(paper["authors"])} authors.')
    for author in paper['authors']:
        print(f'  {author["name"]}')

    print('Finding citing papers with those authors...')
    citations = get_citations(paper_id)
    self_citations = list(filter_self_citing(paper['authors'], citations))
    print(f'Paper has {len(citations)} citations of which {len(self_citations)} contain self-citing authors.')
    for paper, self_citers in self_citations:
        author_names = ', '.join(a['name'] for a in self_citers)
        print(f'  {author_names} in citing paper "{paper["title"]}"')

    print('Finding referenced papers with those authors...')
    references = get_references(paper_id)
    self_references = list(filter_self_citing(paper['authors'], references))
    print(f'Paper has {len(references)} references of which {len(self_references)} contain self-referencing authors.')
    for paper, self_referencers in self_references:
        author_names = ', '.join(a['name'] for a in self_referencers)
        print(f'  {author_names} in referenced paper "{paper["title"]}"')


def get_paper(paper_id):
    rsp = requests.get(f'https://api.semanticscholar.org/graph/v1/paper/{paper_id}',
                       headers={'X-API-KEY': S2_API_KEY},
                       params={'fields': 'title,authors'})
    rsp.raise_for_status()
    return rsp.json()


def get_citations(paper_id):
    edges = get_citation_edges(url=f'https://api.semanticscholar.org/graph/v1/paper/{paper_id}/citations',
                               headers={'X-API-KEY': S2_API_KEY},
                               params={'fields': 'title,authors'})
    return list(edge['citingPaper'] for edge in edges)


def get_references(paper_id):
    edges = get_citation_edges(url=f'https://api.semanticscholar.org/graph/v1/paper/{paper_id}/references',
                               headers={'X-API-KEY': S2_API_KEY},
                               params={'fields': 'title,authors'})
    return list(edge['citedPaper'] for edge in edges)


def get_citation_edges(**req_kwargs):
    """This helps with API endpoints that involve paging."""
    page_size = 1000
    offset = 0
    while True:
        req_kwargs.setdefault('params', dict())
        req_kwargs['params']['limit'] = page_size
        req_kwargs['params']['offset'] = offset
        rsp = requests.get(**req_kwargs)
        rsp.raise_for_status()

        page = rsp.json()["data"]
        for element in page:
            yield element

        if len(page) < page_size:
            break  # no more pages
        offset += page_size


def filter_self_citing(target_authors, papers):
    """Filter papers to those that contain any of the target authors."""
    authors_by_id = {a['authorId']: a for a in target_authors}
    for paper in papers:
        paper_author_ids = {a['authorId'] for a in paper['authors']}
        self_citers = paper_author_ids.intersection(authors_by_id)
        if self_citers:
            yield paper, [authors_by_id[author_id] for author_id in self_citers]


if __name__ == '__main__':
    main()

"A modeling study on freezing characteristics of sessile and pendant water droplets on cold plate surfaces" has 6 authors.
  Xiaoliang Liu
  Lizhen Huang
  Xuan ZHANG
  Long Zhang
  M. Song
  Jun Shen
Finding citing papers with those authors...
Paper has 1 citations of which 1 contain self-citing authors.
  Long Zhang in citing paper "Growth and distribution characteristics of trapped air bubbles in ice slices"
Finding referenced papers with those authors...
Paper has 37 references of which 2 contain self-referencing authors.
  Long Zhang, Mengjie Song in referenced paper "An experimental study on the plate edge effect of frosting characteristics for horizontal flat cold plates with surface wettability considered"
  Keke Shao in referenced paper "Experimental Study on the Effect of Surface Temperature on the Frost Characteristics of an Inverted Cold Plate Under Natural Convection"
